<a href="https://colab.research.google.com/github/yingzibu/drug_design_JAK/blob/main/CoGT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# CoGT (need GPU)

## Works for Conventional and chemberta

## GraphVAE does not work, need to be performed on local env




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !pip install torch
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y

# !pip install torch-geometric==1.7.1
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

!pip install pubchempy --
!pip install transformers
!pip install cairosvg
!pip install varname
!pip install Cython
!pip install rdkit
!pip install molsets
!pip install pathlib
!pip install xgboost==1.6.1

  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 085f2cc64dc18cecc88878b9ae3fa0ec7f7e5739
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 73, in resolve
    collected = self.factory.collect_root_requirements(root_reqs)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 491, in collect_root_requirements
    req = self._make_requ

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-{torch.__version__}.html
  Using cached torch_scatter-2.1.1.tar.gz (107 kB)
  Preparing metadata (setup.py) ... done


In [ ]:
cd /content/drive/MyDrive/JAK_ML

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
from co_model_JAK import *

In [ ]:
smiles = {}
# smiles['test'] = max_len_smi
# smiles['remove_k'] = new
smiles['MMT3-72'] = 'O=C(NCCC(O)=O)C(C=C1)=CC=C1/N=N/C(C=C2C(O)=O)=CC=C2OCCOC3=CC=C(NC4=NC=C(C)C(NC5=CC=CC(S(NC(C)(C)C)(=O)=O)=C5)=N4)C=C3'
smiles['M1'] = 'OCCOC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(N)(=O)=O)=C3)=N2)C=C1'
smiles['M2'] = 'OCCOC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(NC(C)(C)C)(=O)=O)=C3)=N2)C=C1'
smiles['M3'] = 'OC(COC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(NC(C)(C)C)(=O)=O)=C3)=N2)C=C1)=O'
smiles['M4'] = 'O=C(NCCC(O)=O)C(C=C1)=CC=C1/N=N/C(C=C2C(O)=O)=CC=C2OCCOC3=CC=C(NC4=NC=C(C)C(NC5=CC=CC(S(N)(=O)=O)=C5)=N4)C=C3'
smiles['M5'] = 'OC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(NC(C)(C)C)(=O)=O)=C3)=N2)C=C1'


**Conventional models test**

In [ ]:
def simp_model_predict(smi_list, enzyme, ml): # Works for SVM_poly, RF, XGBoost
    if isinstance(smi_list, str):
        smi_list = [smi_list]
    modelname = ml + '_' + enzyme + '.sav'
    model = pickle.load(open(model_path+modelname, 'rb'))
    # maccs = smile_list_to_MACCS(smi)
    pred = model.predict(smile_list_to_MACCS(smi_list))
    prob = model.predict_proba(smile_list_to_MACCS(smi_list))
#     type(prob[0][1])
#     print(ml, pred, prob, prob[0][1])
    return prob[:, 1], pred


# Single smiles
s = smiles['M1']
prob, pred = simp_model_predict(s, 'JAK1', 'SVM_poly')
print('SVM poly:', prob, pred)
prob, pred = simp_model_predict(s, 'JAK1', 'RF')
print('RF:', prob, pred)
prob, pred = simp_model_predict(s, 'JAK1', 'XGBoost')
print('XGBoost: 'prob, pred)

In [ ]:
from function import evaluate
# CAN handle list of smiles
def chembert_predict(smi, enzyme):
    # Turn single smi into list
    if isinstance(smi, str):
        smi = [smi]
    ml = 'chembert'
    file_path = 'model/' + ml + '_' + enzyme + '.pt'

    model = chembert()
    optimizer = optim.AdamW(params=model.parameters(), lr=1e-5, weight_decay=1e-2)
    model.load_state_dict(torch.load(file_path, map_location=torch.device('cuda')))
    model = model.cuda()
    weight_dict = {1: torch.tensor([3.0, 1.0]), 2: torch.tensor([2.0, 1.0]),
                   3: torch.tensor([2.0, 1.0]),4: torch.tensor([2.0, 1.0])}
    params = {'batch_size': 32, 'shuffle': False,
              'drop_last': False, 'num_workers': 0}
    model.eval()

    known_df = pd.DataFrame(smi)
    known_df.columns = ['Smiles']
    known_df['Activity'] = 0
    known_data = jak_dataset_chembert(known_df)
    known_loader = DataLoader(known_data, **params)
    y_pred_list = []
    y_prob_list = []
    X_list = []
    for idx, (X, _) in tqdm(enumerate(known_loader), total=len(known_loader)):
        model.eval()
        output = model(list(X))
        _, y_pred = torch.max(output, 1)
        y_pred_list.extend(y_pred.tolist())
        y_prob_list.extend(torch.softmax(output,1)[:,1].tolist())
        X_list.extend(list(X))


    return y_prob_list, y_pred_list

# chemberta exmaple usage
# works on GPU

# import pandas as pd
# jak1_data = pd.read_csv('data/JAK1_final.csv')
# jak1_data.head()
# from sklearn.model_selection import train_test_split
# from function import evaluate
# train_df, test_df = train_test_split(jak1_data, test_size=0.2, random_state=42)
# probs, preds = chembert_predict(test_df['Smiles'].tolist(), 'JAK1')
# evaluate(test_df['Activity'].tolist(), preds, probs)

chembert_predict(list(smiles.values())[:4], 'JAK1')

In [ ]:
smi = ['COOC', 'ClCOP', 'CON']
type(prob.tolist())


list

In [ ]:
prob, pred = simp_model_predict(smi, 'JAK1', 'RF')
prob, pred

(array([0.20754717, 0.19622642, 0.18867925]), array([0, 0, 0]))

In [ ]:

type(test_df['Smiles'].tolist())

list

In [ ]:
prob_jak, pred_jak = simp_model_predict(test_df['Smiles'].tolist(), 'JAK2', 'XGBoost')
evaluate(test_df['Activity'].tolist(), pred_jak, prob_jak)

Accuracy, weighted accuracy, precision, recall/SE, SP,     F1,     AUC,     MCC,     AP
& 0.912  &  0.889  &          0.913  &     0.960  &0.817  &0.936 &0.959 &   0.801 &   0.976


In [ ]:
import pandas as pd
jak1_data = pd.read_csv('data/JAK2_final.csv')
jak1_data.head()
from sklearn.model_selection import train_test_split
from function import evaluate
train_df, test_df = train_test_split(jak1_data, test_size=0.2, random_state=42)
probs, preds = chembert_predict(test_df['Smiles'].tolist(), 'JAK2')
print()
evaluate(test_df['Activity'].tolist(), preds, probs)

100%|██████████| 64/64 [00:04<00:00, 13.95it/s]


Accuracy, weighted accuracy, precision, recall/SE, SP,     F1,     AUC,     MCC,     AP
& 0.897  &  0.893  &          0.939  &     0.903  &0.884  &0.921 &0.957 &   0.773 &   0.976


In [ ]:
# def GVAE_predict(smi, enzyme, model_path=model_path, device='cpu'):
#     if isinstance(smi, str):
#         smi = [smi]
#     smiles, nodes, edges, relations = preprocess_test(smi)
#     y = [0]*len(smiles)
# #     print(len(smiles))

#     test_set = GDataset(nodes, edges, relations,y, range(len(smiles)))
#     test_loader = DataLoader(test_set, batch_size=len(smiles), shuffle=False)

#     model = torch.load(model_path+'GVAE'+ '_' + enzyme + '.pt')
# #     print(model_path+'GVAE'+ '_' + enzyme + '.pt')
#     model.eval()
#     for data in test_loader:
#         data.to(device)
#         preds = model(data.x, data.edge_index, data.edge_type, data.batch, 'fintune')
# #         print(preds)
# #         print(get_preds(preds)[0])

#     return preds, get_preds(preds)[0]
# GVAE_predict(smi, 'JAK2', 'model/')